In [1]:
from pandas import DataFrame,read_csv,read_excel,merge,melt,pivot_table
import numpy
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sb
import sys
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from factor_analyzer.factor_analyzer import calculate_kmo

## 데이터 준비

In [2]:
origin=read_excel('행정동병합테스트22(개발제한행정동제외).xlsx')
origin.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,노인인구(65세이상),장애인_인구,세대당공원면적,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,인구밀도
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,2154,437,337.55,4,0.352898,0,32,0,3,4552.529183
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,1808,267,60.79,0,0.063554,0,27,0,12,7611.382114
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,626,110,956.51,3,1.000000,0,16,0,5,1728.859060
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,1814,311,0.00,3,0.000000,10,34,0,5,4130.837004
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,3495,491,1.14,3,0.001192,10,54,0,5,1990.755355


In [3]:
origin.columns

Index(['시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명', '생성일자', '말소일자',
       '지하철역_수', '의료기관_수', '사회복지시설_수', '계(명)', '면적 (k㎡)', '구성비 (%)',
       '행정동별_공원수', '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수', '개소 (공영+민영)',
       '면수 (공영+민영)', '하천및한강여부', '아동인구(0~9세)', '노인인구(65세이상)', '장애인_인구',
       '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수', '행정동별버스정류장수', '행정동별유통업체수',
       '공공자전거대여소수', '인구밀도'],
      dtype='object')

In [4]:
origin.isna().sum()
origin1=origin.fillna(0)

## 결측치는 0으로 처리

> 실제 조사된 값이므로 없다면 관측되지 아니함으로 판단

In [5]:
origin1.columns

Index(['시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명', '생성일자', '말소일자',
       '지하철역_수', '의료기관_수', '사회복지시설_수', '계(명)', '면적 (k㎡)', '구성비 (%)',
       '행정동별_공원수', '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수', '개소 (공영+민영)',
       '면수 (공영+민영)', '하천및한강여부', '아동인구(0~9세)', '노인인구(65세이상)', '장애인_인구',
       '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수', '행정동별버스정류장수', '행정동별유통업체수',
       '공공자전거대여소수', '인구밀도'],
      dtype='object')

## 데이터 스케일링

In [6]:
scaler=MinMaxScaler()
df = origin1.copy()
df['의료기관minmax']=scaler.fit_transform(df[['의료기관_수']])
df['사회복지minmax']=scaler.fit_transform(df[['사회복지시설_수']])
df['보육시설minmax']=scaler.fit_transform(df[['보육시설수']])
df['노인인구minmax']=scaler.fit_transform(df[['노인인구(65세이상)']])
df['아동인구minmax']=scaler.fit_transform(df[['아동인구(0~9세)']])
df['장애인minmax']=scaler.fit_transform(df[['장애인_인구']])
df.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,인구밀도,의료기관minmax,사회복지minmax,보육시설minmax,노인인구minmax,아동인구minmax,장애인minmax
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,32,0,3,4552.529183,0.017188,0.185185,0.121212,0.245223,0.094081,0.119035
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,27,0,12,7611.382114,0.117188,0.037037,0.000000,0.205630,0.075019,0.072408
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,16,0,5,1728.859060,0.017188,0.000000,0.090909,0.070374,0.016603,0.029347
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,34,0,5,4130.837004,0.006250,0.148148,0.090909,0.206317,0.059185,0.084476
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,54,0,5,1990.755355,0.023438,0.407407,0.090909,0.398673,0.158801,0.133845


## 파생변수(`사회적약자_지원_인프라_밀집도`) 생성

In [8]:
df['사회적약자_지원_인프라_밀집도'] = (df['의료기관minmax'] + df['사회복지minmax'] + df['보육시설minmax']) / (df['노인인구minmax'] + df['아동인구minmax'] + df['장애인minmax'])
df.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,행정동별유통업체수,공공자전거대여소수,인구밀도,의료기관minmax,사회복지minmax,보육시설minmax,노인인구minmax,아동인구minmax,장애인minmax,사회적약자_지원_인프라_밀집도
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,0,3,4552.529183,0.017188,0.185185,0.121212,0.245223,0.094081,0.119035,0.705995
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,0,12,7611.382114,0.117188,0.037037,0.000000,0.205630,0.075019,0.072408,0.436826
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,0,5,1728.859060,0.017188,0.000000,0.090909,0.070374,0.016603,0.029347,0.929271
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,0,5,4130.837004,0.006250,0.148148,0.090909,0.206317,0.059185,0.084476,0.700922
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,0,5,1990.755355,0.023438,0.407407,0.090909,0.398673,0.158801,0.133845,0.754723


## 파생 변수의 값 정렬

In [9]:
df['사회적약자_지원_인프라_밀집도'].value_counts().sort_index()

사회적약자_지원_인프라_밀집도
0.011184     1
0.039567     1
0.058412     1
0.209584     1
0.219126     1
            ..
2.108114     1
3.374451     1
4.537069     1
10.164062    1
12.600828    1
Name: count, Length: 406, dtype: int64

## 결측치 확인

In [10]:
df['사회적약자_지원_인프라_밀집도'].isna().sum()

0

In [10]:
# df['사회적약자지수'].filln(0)
# df['사회적약자지수'].isnu().sum()

In [11]:
df['사회적약자_지원_인프라_밀집도'].unique()

array([7.05995102e-01, 4.36825837e-01, 9.29271388e-01, 7.00922099e-01,
       7.54722656e-01, 4.01112846e-01, 5.99942335e-01, 8.03755062e-01,
       2.10811382e+00, 1.84524943e+00, 1.24161501e+00, 4.57867092e-01,
       6.40084078e-01, 2.83573952e-01, 2.19126111e-01, 6.20639747e-01,
       4.39999915e-01, 3.37445134e+00, 1.59168945e+00, 4.53706883e+00,
       1.22944179e+00, 4.17028999e-01, 5.32485213e-01, 1.97129531e+00,
       3.30896319e-01, 5.54736774e-01, 5.01124170e-01, 3.53360132e-01,
       4.38746017e-01, 4.35573548e-01, 4.34565371e-01, 9.36593061e-01,
       8.44264681e-01, 2.62772949e-01, 1.40090657e+00, 5.59877224e-01,
       7.68054935e-01, 7.20374043e-01, 1.41124107e+00, 5.75436533e-01,
       5.71105760e-01, 4.13097776e-01, 4.47451572e-01, 1.06960258e+00,
       3.10979971e-01, 5.74785703e-01, 4.69750561e-01, 3.51119950e-01,
       5.67045133e-01, 9.66270163e-01, 8.05838099e-01, 6.33347539e-01,
       7.61771049e-01, 4.74901784e-01, 3.97999629e-01, 6.03780534e-01,
      

In [12]:
df.columns

Index(['시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명', '생성일자', '말소일자',
       '지하철역_수', '의료기관_수', '사회복지시설_수', '계(명)', '면적 (k㎡)', '구성비 (%)',
       '행정동별_공원수', '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수', '개소 (공영+민영)',
       '면수 (공영+민영)', '하천및한강여부', '아동인구(0~9세)', '노인인구(65세이상)', '장애인_인구',
       '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수', '행정동별버스정류장수', '행정동별유통업체수',
       '공공자전거대여소수', '인구밀도', '의료기관minmax', '사회복지minmax', '보육시설minmax',
       '노인인구minmax', '아동인구minmax', '장애인minmax', '사회적약자_지원_인프라_밀집도'],
      dtype='object')

## 지수 파일 생성에 사용될 컬럼 추출

In [13]:
df1=df.filter(['시군구코드','시군구명','행정동코드','행정동코드수정','읍면동명','의료기관_수','사회복지시설_수','보육시설수', '노인인구(65세이상)','아동인구(0~9세)','장애인_인구', '의료기관minmax', '사회복지minmax', '보육시설minmax', '노인인구minmax', '아동인구minmax', '장애인minmax','사회적약자_지원_인프라_밀집도'])
df1.head()

,시군구코드,시군구명,행정동코드,읍면동명,의료기관_수,사회복지시설_수,보육시설수,노인인구(65세이상),아동인구(0~9세),장애인_인구,의료기관minmax,사회복지minmax,보육시설minmax,노인인구minmax,아동인구minmax,장애인minmax,사회적약자_지원_인프라_밀집도
0,11110,종로구,1111051500,청운효자동,11,5,4,2154,612,437,0.017188,0.185185,0.121212,0.245223,0.094081,0.119035,0.705995
1,11110,종로구,1111053000,사직동,75,1,0,1808,488,267,0.117188,0.037037,0.000000,0.205630,0.075019,0.072408,0.436826
2,11110,종로구,1111054000,삼청동,11,0,3,626,108,110,0.017188,0.000000,0.090909,0.070374,0.016603,0.029347,0.929271
3,11110,종로구,1111055000,부암동,4,4,3,1814,385,311,0.006250,0.148148,0.090909,0.206317,0.059185,0.084476,0.700922
4,11110,종로구,1111056000,평창동,15,11,3,3495,1033,491,0.023438,0.407407,0.090909,0.398673,0.158801,0.133845,0.754723


## 지수 파일 생성

In [14]:
df1.to_excel('사회적약자_지원_인프라_밀집도1.xlsx', index=False)

In [15]:
df2=read_excel('사회적약자_지원_인프라_밀집도1.xlsx')
df2.head()

,시군구코드,시군구명,행정동코드,읍면동명,의료기관_수,사회복지시설_수,보육시설수,노인인구(65세이상),아동인구(0~9세),장애인_인구,의료기관minmax,사회복지minmax,보육시설minmax,노인인구minmax,아동인구minmax,장애인minmax,사회적약자_지원_인프라_밀집도
0,11110,종로구,1111051500,청운효자동,11,5,4,2154,612,437,0.017188,0.185185,0.121212,0.245223,0.094081,0.119035,0.705995
1,11110,종로구,1111053000,사직동,75,1,0,1808,488,267,0.117188,0.037037,0.000000,0.205630,0.075019,0.072408,0.436826
2,11110,종로구,1111054000,삼청동,11,0,3,626,108,110,0.017188,0.000000,0.090909,0.070374,0.016603,0.029347,0.929271
3,11110,종로구,1111055000,부암동,4,4,3,1814,385,311,0.006250,0.148148,0.090909,0.206317,0.059185,0.084476,0.700922
4,11110,종로구,1111056000,평창동,15,11,3,3495,1033,491,0.023438,0.407407,0.090909,0.398673,0.158801,0.133845,0.754723


## 이상치 `inf`, `-inf` 처리

In [16]:
# DataFrame(df2)에서 '사회적약자지수' 열을 선택
accessibility_series = df2['사회적약자_지원_인프라_밀집도']

# 'inf'와 '-inf'를 0으로 대체
accessibility_series = accessibility_series.replace([np.inf, -np.inf], 0)

# 변경된 '사회적약자지수' 열을 DataFrame에 할당
df2['사회적약자_지원_인프라_밀집도'] = accessibility_series

In [17]:
df2.to_excel("사회적약자_지원_인프라_밀집도1.xlsx", index=False)

In [18]:
df2 = read_excel("사회적약자_지원_인프라_밀집도1.xlsx")
df2.head()

,시군구코드,시군구명,행정동코드,읍면동명,의료기관_수,사회복지시설_수,보육시설수,노인인구(65세이상),아동인구(0~9세),장애인_인구,의료기관minmax,사회복지minmax,보육시설minmax,노인인구minmax,아동인구minmax,장애인minmax,사회적약자_지원_인프라_밀집도
0,11110,종로구,1111051500,청운효자동,11,5,4,2154,612,437,0.017188,0.185185,0.121212,0.245223,0.094081,0.119035,0.705995
1,11110,종로구,1111053000,사직동,75,1,0,1808,488,267,0.117188,0.037037,0.000000,0.205630,0.075019,0.072408,0.436826
2,11110,종로구,1111054000,삼청동,11,0,3,626,108,110,0.017188,0.000000,0.090909,0.070374,0.016603,0.029347,0.929271
3,11110,종로구,1111055000,부암동,4,4,3,1814,385,311,0.006250,0.148148,0.090909,0.206317,0.059185,0.084476,0.700922
4,11110,종로구,1111056000,평창동,15,11,3,3495,1033,491,0.023438,0.407407,0.090909,0.398673,0.158801,0.133845,0.754723


## 지수 정규화

In [19]:
df2['사회적약자_지원_인프라_밀집도minmax'] = scaler.fit_transform(df2[['사회적약자_지원_인프라_밀집도']])
df2

,시군구코드,시군구명,행정동코드,읍면동명,의료기관_수,사회복지시설_수,보육시설수,노인인구(65세이상),아동인구(0~9세),장애인_인구,의료기관minmax,사회복지minmax,보육시설minmax,노인인구minmax,아동인구minmax,장애인minmax,사회적약자_지원_인프라_밀집도,사회적약자_지원_인프라_밀집도minmax
0,11110,종로구,1111051500,청운효자동,11,5,4,2154,612,437,0.017188,0.185185,0.121212,0.245223,0.094081,0.119035,0.705995,0.055189
1,11110,종로구,1111053000,사직동,75,1,0,1808,488,267,0.117188,0.037037,0.000000,0.205630,0.075019,0.072408,0.436826,0.033809
2,11110,종로구,1111054000,삼청동,11,0,3,626,108,110,0.017188,0.000000,0.090909,0.070374,0.016603,0.029347,0.929271,0.072924
3,11110,종로구,1111055000,부암동,4,4,3,1814,385,311,0.006250,0.148148,0.090909,0.206317,0.059185,0.084476,0.700922,0.054786
4,11110,종로구,1111056000,평창동,15,11,3,3495,1033,491,0.023438,0.407407,0.090909,0.398673,0.158801,0.133845,0.754723,0.059060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,11740,강동구,1174065000,성내제2동,62,6,9,4469,897,1075,0.096875,0.222222,0.272727,0.510127,0.137894,0.294021,0.628236,0.049013
402,11740,강동구,1174066000,성내제3동,80,6,11,4098,1024,913,0.125000,0.222222,0.333333,0.467674,0.157417,0.249589,0.778063,0.060913
403,11740,강동구,1174068500,길동,103,11,17,8478,2061,1988,0.160938,0.407407,0.515152,0.968875,0.316833,0.544432,0.592029,0.046137
404,11740,강동구,1174069000,둔촌제1동,1,0,0,11,1,3,0.001563,0.000000,0.000000,0.000000,0.000154,0.000000,10.164062,0.806447


## 최종 데이터 저장

In [21]:
df2.to_excel('사회적약자_지원_인프라_밀집도1.xlsx',index=False)